# GPX Viewer

In [ ]:
import datetime
import json
from collections import defaultdict
from io import StringIO
from statistics import mean

import gpxpy
import srtm
import pandas as pd

from ipyleaflet import Map, Polyline, Marker
from ipyupload import FileUpload 
from ipywidgets import HTML, HBox, Label, GridBox, Output, IntSlider, Layout

In [ ]:
data_viewer = GridBox(
    children=[],
    layout=Layout(
        width='100%',
        grid_template_rows='auto auto',
        grid_template_columns='25% 25% 25% 25%',
        grid_template_areas='''
        "header header header header"
        "main main main main"
        ''')
   )

In [ ]:
def parse_data(file):
    gpx = gpxpy.parse(file)
    elevation_data = srtm.get_data()
    elevation_data.add_elevations(gpx, smooth=True)
    return gpx

In [ ]:
def plot_map(gpx):
    points = [p.point for p in gpx.get_points_data(distance_2d=True)]
    mean_lat = mean(p.latitude for p in points)
    mean_lng = mean(p.longitude for p in points)

    # create map
    m = Map(center=(mean_lat, mean_lng), zoom=15)

    # show trace
    line = Polyline(locations=[[[p.latitude, p.longitude] for p in points],],
                    color = "red", fill=False)
    m.add_layer(line)

    # add markers
    for point in gpx.waypoints:
        lat, lng = point.latitude, point.longitude
        marker = Marker(location=(lat, lng), title=point.name, popup=HTML(value=point.name), draggable=False)
        m.add_layer(marker);

    return HBox([m], layout=Layout(width='100%', grid_area='main'))

In [ ]:
def plot_stats(gpx):
    lowest, highest = gpx.get_elevation_extremes()
    uphill, downhill = gpx.get_uphill_downhill()
    points = gpx.get_points_data(distance_2d=True)
    
    _, distance_from_start, *rest = points[-1]
    
    stats = defaultdict(list)
    stats['Date'].append(gpx.get_time_bounds().start_time.strftime("%Y-%m-%d"))
    stats['Distance'].append(round(distance_from_start / 1000, 2))
    stats['Duration'].append(str(datetime.timedelta(seconds=gpx.get_duration())))
    stats['Lowest'].append(int(lowest))
    stats['Highest'].append(int(highest))
    stats['Uphill'].append(int(uphill))
    stats['Downhill'].append(int(downhill))
    
    df = pd.DataFrame(
        stats,
        columns=['Date', 'Distance', 'Duration', 'Lowest', 'Highest', 'Uphill', 'Downhill']
    )
    
    return HTML(value=df.to_html(index=False),
               layout=Layout(width='100%', grid_area='header'))

In [ ]:
def parse_files(change):
    # unpack data
    all_files = [v for _, v in change['new'].items()]
    raw = all_files[0]['content'].decode('utf-8')
    with StringIO(raw) as f:
        gpx = parse_data(f)
    
    stats = plot_stats(gpx)
    m = plot_map(gpx)

    data_viewer.children = [stats, m]

In [ ]:
uploader = FileUpload(accept=None,
           multiple=False, 
           disabled=False,
           style_button='color: darkblue; background-color: lightsalmon; width: 180px;',
           compress_level=9
          )

uploader.observe(parse_files, names='value')

In [ ]:
uploader

In [ ]:
data_viewer